In [ ]:
"""This is a demo VQPy implementation listing and storing all red moving
vehicles to a json file."""

from typing import Optional
import argparse
import numpy as np
import vqpy
import cv2

@vqpy.vqpy_func_logger(['image'], ['major_color_rgb'], [], required_length=1)
def get_image_color(obj, image: Optional[np.ndarray]) -> str:
    import webcolors
    from colordetect import ColorDetect

    class BindedColorDetect(ColorDetect):
        def _find_unique_colors(self, cluster, centroids) -> dict:
            labels = np.arange(0, len(np.unique(cluster.labels_)) + 1)
            (hist, _) = np.histogram(cluster.labels_, bins=labels)
            hist = hist.astype("float")
            hist /= hist.sum()

            # iterate through each cluster's color and percentage
            colors = sorted(
                [(percent * 100, color)
                 for (percent, color) in zip(hist, centroids)],
                key=lambda x: (x[0], x[1].tolist())
            )

            for (percent, color) in colors:
                color.astype("uint8").tolist()
            return dict(colors)

    if image is None:
        return [None]

    maximum_width = 16
    ratio = max(maximum_width / image.shape[0], maximum_width / image.shape[1])
    if ratio < 1:
        size = (int(image.shape[0] * ratio + 0.5),
                int(image.shape[1] * ratio + 0.5))
        image = cv2.resize(image, size, interpolation=cv2.INTER_LINEAR)
    detector = BindedColorDetect(image)
    result = detector.get_color_count()
    bestrgb, bestp = None, 0
    for color, percent in result.items():
        rgb = webcolors.name_to_rgb(color)
        if max(*rgb) - min(*rgb) <= 15:
            percent /= 5
        if percent > bestp:
            bestrgb, bestp = rgb, percent
    return [bestrgb]

class Vehicle(vqpy.VObjBase):
    """The class of vehicles"""
    required_fields = ['class_id', 'tlbr']

    @vqpy.property()
    @vqpy.postproc({'majority': 100})
    def license_plate(self):
        """The license plate of the vehicle"""
        return self.infer('license_plate', {'license_plate': 'openalpr'})

In [ ]:
# ~ 1min
import json
import webcolors

class ListMovingVehicle(vqpy.QueryBase):
    """The class obtaining all moving vehicles"""
    @staticmethod
    def setting() -> vqpy.VObjConstraint:
        filter_cons = {'__class__': lambda x: x == Vehicle,
                       'bbox_velocity': lambda x: x >= 0.1}
        select_cons = {'track_id': None,
                       'license_plate': None}
        return vqpy.VObjConstraint(filter_cons=filter_cons,
                                   select_cons=select_cons,
                                   filename='moving')

class ListMovingVehicleWithPlate(ListMovingVehicle):
    @staticmethod
    def setting() -> vqpy.VObjConstraint:
        filter_cons = {'image': lambda x: x is not None,
                       'license_plate': lambda x: x is not None}
        select_cons = {'track_id': None,
                       'license_plate': None,
                       'class_id': lambda x: vqpy.COCO_CLASSES[x],
                       'major_color_rgb': webcolors.rgb_to_name}
        return vqpy.VObjConstraint(filter_cons=filter_cons,
                                   select_cons=select_cons,
                                   filename='moving')

vqpy.launch(cls_name=vqpy.COCO_CLASSES,
            cls_type={"car": Vehicle, "truck": Vehicle},
            tasks=[ListMovingVehicleWithPlate()],
            video_path='videos/demo.mp4',
            save_folder='vqpy_outputs/demo')

In [ ]:
data = json.load(open('vqpy_outputs/demo/moving.json'))
print(json.dumps(data, indent=4))

In [ ]:
# ~1.5min

class ListRedMovingCar(ListMovingVehicle):
    """The class obtaining all red moving vehicles"""
    @staticmethod
    def setting() -> vqpy.VObjConstraint:
        import webcolors

        def rgb_is_red(color):
            if color is None:
                return False
            color = np.asarray(color)
            return color[0] ** 2 > sum(color * color) * 0.7

        filter_cons = {'major_color_rgb': rgb_is_red}
        select_cons = {'track_id': None,
                       'license_plate': None,
                       'class_id': lambda x: vqpy.COCO_CLASSES[x],
                       'major_color_rgb': webcolors.rgb_to_name}
        return vqpy.VObjConstraint(select_cons=select_cons,
                                   filter_cons=filter_cons,
                                   filename="redmoving")

vqpy.launch(cls_name=vqpy.COCO_CLASSES,
            cls_type={"car": Vehicle},
            tasks=[ListRedMovingCar()],
            video_path='videos/demo.mp4',
            save_folder='vqpy_outputs/demo')

In [ ]:
data = json.load(open('vqpy_outputs/demo/redmoving.json'))
print(json.dumps(data, indent=4))

In [ ]:
data = json.load(open('vqpy_outputs/count_person/count.json'))
print(json.dumps(data, indent=4))

In [ ]:
print(open('examples/count_person/main.py', 'r').read())